In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
import keras
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, RocCurveDisplay, auc, roc_curve, confusion_matrix, ConfusionMatrixDisplay

In [3]:
dataset = pd.read_csv('/Users/aisiri/Desktop/DS 340W/Parentpaper1/creditcard.csv',index_col='Time')
dataset.isnull().sum()

V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [4]:
normal=dataset[dataset.Class == 0]
fraud =dataset[dataset.Class == 1]

print(normal.shape)
print(fraud.shape)

(284315, 30)
(492, 30)


In [5]:
normal_sample=normal.sample(n=492)
new_dataset=pd.concat([normal_sample, fraud], axis=0)

X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)
print(X.shape, X_train.shape, X_test.shape)

(984, 29) (787, 29) (197, 29)


## Models
We have used 4 models for our project.

1. Ridge Classification
2. Logistic Regression
3. Random Forest
4. Decision Tree

# RIDGE CLASSIFICATION


In [6]:
ridge = RidgeClassifier()

scores = cross_validate(ridge, X_train,Y_train, scoring=['precision', 'recall', 'f1'], n_jobs=-1)
print("Cross validation precision: mean={0}, std={1}".format(np.mean(scores['test_precision']), np.std(scores['test_precision'])))
print("Cross validation recall: mean={0}, std={1}".format(np.mean(scores['test_recall']), np.std(scores['test_recall'])))
print("Cross validation f1 score: mean={0}, std={1}".format(np.mean(scores['test_f1']), np.std(scores['test_f1'])))  

Cross validation precision: mean=0.9684578845622756, std=0.01252421661331967
Cross validation recall: mean=0.8478740668614086, std=0.06020554634478506
Cross validation f1 score: mean=0.9028403150228916, std=0.03455222369085763


In [7]:
ridge = RidgeClassifier()
ridge.fit(X_train, Y_train)
ridge_predictions = ridge.predict(X_test)
ridge_precision = precision_score(Y_test, ridge_predictions)
ridge_recall = recall_score(Y_test, ridge_predictions)
print(f'Ridge Classifier Precision: {ridge_precision}')
print(f'Ridge Classifier Recall: {ridge_recall}')

Ridge Classifier Precision: 0.9873417721518988
Ridge Classifier Recall: 0.7959183673469388


# Logistic Regression

In [9]:

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Create an instance of the LogisticRegression model
model = LogisticRegression()


# Perform cross-validation and compute the accuracy score
accuracy_scores = cross_val_score(model, X_train, Y_train, cv=3, n_jobs=-1, scoring='accuracy')

# Print the mean accuracy across the cross-validation folds
mean_accuracy = accuracy_scores.mean()
print(f'Mean accuracy: {mean_accuracy}')

params_grid_LR = {"solver":['newton-cg', 'lbfgs', 'liblinear'],
               "penalty":['l2'],"C":[100, 10, 1.0, 0.1, 0.01]}

grid_search_LR = GridSearchCV(LogisticRegression(random_state=42),params_grid_LR,n_jobs=-1,cv=3,scoring='accuracy')
grid_search_LR.fit(X_train,Y_train)
grid_search_LR.best_params_
LR = grid_search_LR.best_estimator_
acc_LR = accuracy_score(Y_test,LR.predict(X_test))
print('Logistic Regression:\n','Train accuracy:{}\n'.format(round(grid_search_LR.best_score_,4)),
     'Test accuracy: {}'.format(round(acc_LR,4)))



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

Mean accuracy: 0.9326425371762498
Logistic Regression:
 Train accuracy:0.9377
 Test accuracy: 0.9239


# RANDON FOREST

In [10]:
rfmodel = RandomForestClassifier(n_jobs=-1, n_estimators=200)

scores = cross_validate(rfmodel, X_train, Y_train, scoring=['precision', 'recall', 'f1'], n_jobs=-1)
print("Cross validation precision: mean={0}, std={1}".format(np.mean(scores['test_precision']), np.std(scores['test_precision'])))
print("Cross validation recall: mean={0}, std={1}".format(np.mean(scores['test_recall']), np.std(scores['test_recall'])))
print("Cross validation f1 score: mean={0}, std={1}".format(np.mean(scores['test_f1']), np.std(scores['test_f1'])))

rfmodel = RandomForestClassifier(n_jobs=-1, n_estimators=200)
rfmodel.fit(X_train, Y_train)
rf_predictions = rfmodel.predict(X_test)
rf_precision = precision_score(Y_test, rf_predictions)
rf_recall = recall_score(Y_test, rf_predictions)
print(f'Random Forest Precision for Testing data: {rf_precision}')
print(f'Random Forest Recall for testinf data: {rf_recall}')



Cross validation precision: mean=0.9707666729405859, std=0.02210535232058248
Cross validation recall: mean=0.8884777669587797, std=0.039346445882852195
Cross validation f1 score: mean=0.9268663394280526, std=0.01505335048163393
Random Forest Precision for Testing data: 0.9882352941176471
Random Forest Recall for testinf data: 0.8571428571428571


# DECISION TREE

In [12]:
dtmodel = DecisionTreeClassifier(min_samples_split=3)

scores = cross_validate(dtmodel, X_train, Y_train, scoring=['precision', 'recall', 'f1'], n_jobs=-1)
print("Cross validation precision: mean={0}, std={1}".format(np.mean(scores['test_precision']), np.std(scores['test_precision'])))
print("Cross validation recall: mean={0}, std={1}".format(np.mean(scores['test_recall']), np.std(scores['test_recall'])))
print("Cross validation f1 score: mean={0}, std={1}".format(np.mean(scores['test_f1']), np.std(scores['test_f1'])))  


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Create an instance of the LogisticRegression model
model = DecisionTreeClassifier(random_state=42,max_depth=5)

# Perform cross-validation and compute the accuracy score
accuracy_scores = cross_val_score(model, X_train, Y_train, cv=3, n_jobs=-1, scoring='accuracy')

# Print the mean accuracy across the cross-validation folds
mean_accuracy = accuracy_scores.mean()
print(f'Mean accuracy for testing data: {mean_accuracy}')

Cross validation precision: mean=0.9211870212734677, std=0.02820019050476263
Cross validation recall: mean=0.9011359948068808, std=0.02909320136980504
Cross validation f1 score: mean=0.9102474283033877, std=0.008704517920562429
Mean accuracy for testing data: 0.905992705037781


In [11]:
# WITH HYPERPARAMETER TUNING

params_grid_DecisionTree = {"max_features":['sqrt', 'log2'],
               "max_depth":[3,5,10,15,20]}
grid_search_DT = GridSearchCV(DecisionTreeClassifier(random_state=42),params_grid_DecisionTree,n_jobs=-1,
                           cv=3,scoring='accuracy')
grid_search_DT.fit(X_train,Y_train)
grid_search_DT.best_params_
DT = grid_search_DT.best_estimator_
acc_DT = accuracy_score(Y_test,DT.predict(X_test))
print('Decision Tree:\n','Train accuracy:{}\n'.format(round(grid_search_DT.best_score_,4)),
     'Test accuracy: {}'.format(round(acc_DT,4)))

Decision Tree:
 Train accuracy:0.9098
 Test accuracy: 0.9137


# COMPARING USING AUC - ROC CURVE